In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,OrdinalEncoder,OneHotEncoder
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.model_selection import GridSearchCV,train_test_split,cross_val_score,RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
import xgboost as xgb
import seaborn as sns
import random
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/marketing-strategy-personalised-offer/sample.csv
/kaggle/input/marketing-strategy-personalised-offer/train_data.csv
/kaggle/input/marketing-strategy-personalised-offer/test_data.csv


In [2]:
Model_Scores={}

In [3]:
demo=pd.read_csv('/kaggle/input/marketing-strategy-personalised-offer/train_data.csv')
demo_y=demo.pop('Offer Accepted')

In [4]:
ordinal_col_to_change=['offer expiration','income_range','no_visited_Cold drinks','Restaur_spend_less_than20',
                      'age','no_visited_bars','no_Take-aways','Restaur_spend_greater_than20']
nominal_col_to_change=['Marital Status','restaurant type','gender','car','Customer type','Qualification','Job/Job Industry','Climate','drop location']
col_to_change=nominal_col_to_change+ordinal_col_to_change
demo.pop('travelled_more_than_5mins_for_offer')

0        1
1        1
2        1
3        1
4        1
        ..
12374    1
12375    1
12376    1
12377    1
12378    1
Name: travelled_more_than_5mins_for_offer, Length: 12379, dtype: int64

In [5]:
imputer = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
imputer2 = SimpleImputer(missing_values=np.nan,strategy='mean')
for i in demo.columns:
    if demo[i].dtypes=="object":
        demo[i]=imputer.fit_transform(np.array(demo[i]).reshape(-1,1))
    if demo[i].dtypes=="int64":
        demo[i]=imputer2.fit_transform(np.array(demo[i]).reshape(-1,1))

In [6]:
ordenc=OrdinalEncoder()
ordinal_col_to_change=['offer expiration','income_range','no_visited_Cold drinks','Restaur_spend_less_than20',
                      'age','no_visited_bars','no_Take-aways','Restaur_spend_greater_than20']
nominal_col_to_change=['Marital Status','restaurant type','gender','car','Customer type','Qualification','Job/Job Industry','Climate','drop location']
for i in demo.columns:
    if demo[i].dtypes=="object":
        demo[i]=ordenc.fit_transform(np.array(demo[i]).reshape(-1,1))
demo_y=demo_y.replace(to_replace='Yes',value=1)
demo_y=demo_y.replace(to_replace='No',value=0)

In [7]:
demo=StandardScaler().fit_transform(demo)

In [8]:
X_train,X_test,y_train,y_test=train_test_split(demo,demo_y,test_size=0.2,random_state=64)

In [9]:
"""dc = DummyClassifier(strategy='uniform', random_state=64)
dc.fit(X_train, y_train)
Model_Scores["DummyClassifier"]=f1_score(y_test,dc.predict(X_test))"""

'dc = DummyClassifier(strategy=\'uniform\', random_state=64)\ndc.fit(X_train, y_train)\nModel_Scores["DummyClassifier"]=f1_score(y_test,dc.predict(X_test))'

In [10]:
"""param_grid={'penalty':['l2'],
           'C':np.logspace(-4,4,20),
           'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
           'max_iter':[1000,1500,2000]}
Best parameter values:'C': 0.03359818286283781, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'lbfgs'
lr=LogisticRegression(max_iter=1000,C=0.03359818286283781,penalty='l2',solver='lbfgs')
lr.fit(X_train,y_train)
Model_Scores["LogisticRegression"]=f1_score(y_test,lr.predict(X_test))"""

'param_grid={\'penalty\':[\'l2\'],\n           \'C\':np.logspace(-4,4,20),\n           \'solver\':[\'newton-cg\', \'lbfgs\', \'liblinear\', \'sag\', \'saga\'],\n           \'max_iter\':[1000,1500,2000]}\nBest parameter values:\'C\': 0.03359818286283781, \'max_iter\': 1000, \'penalty\': \'l2\', \'solver\': \'lbfgs\'\nlr=LogisticRegression(max_iter=1000,C=0.03359818286283781,penalty=\'l2\',solver=\'lbfgs\')\nlr.fit(X_train,y_train)\nModel_Scores["LogisticRegression"]=f1_score(y_test,lr.predict(X_test))'

In [11]:
"""param_grid={'alpha':list(np.arange(0,2,0.01))} 
Best parameter value:{'alpha':0.0}
meaning the model performs better without any regularization so its basically linear regression
rs=RidgeClassifier(alpha=0.0) 
rs.fit(X_train,y_train)
Model_Scores["RidgeClassifier"]=f1_score(y_test,rs.predict(X_test))"""

'param_grid={\'alpha\':list(np.arange(0,2,0.01))} \nBest parameter value:{\'alpha\':0.0}\nmeaning the model performs better without any regularization so its basically linear regression\nrs=RidgeClassifier(alpha=0.0) \nrs.fit(X_train,y_train)\nModel_Scores["RidgeClassifier"]=f1_score(y_test,rs.predict(X_test))'

In [12]:
"""param_grid={'var_smoothing': np.logspace(0,-9, num=100)}
Best parameter value:{'var_smoothing': 0.02310129700083159}
gnb=GaussianNB(var_smoothing=0.02310129700083159) #using gaussian naive bayes because 
#it works well with continous values assuming data is normally distributed 
#which it should since its normalised
gnb.fit(X_train,y_train)
Model_Scores["GaussianNaiveBayes"]=f1_score(y_test,gnb.predict(X_test))"""

'param_grid={\'var_smoothing\': np.logspace(0,-9, num=100)}\nBest parameter value:{\'var_smoothing\': 0.02310129700083159}\ngnb=GaussianNB(var_smoothing=0.02310129700083159) #using gaussian naive bayes because \n#it works well with continous values assuming data is normally distributed \n#which it should since its normalised\ngnb.fit(X_train,y_train)\nModel_Scores["GaussianNaiveBayes"]=f1_score(y_test,gnb.predict(X_test))'

In [13]:
"""grid_params = { 'n_neighbors' : list(range(1,21,1)),
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}
Best parameter value:{'metric': 'manhattan', 'n_neighbors': 18, 'weights': 'distance'}
knc=KNeighborsClassifier(metric="manhattan",n_neighbors=18,weights="distance")
knc.fit(X_train,y_train)
Model_Scores["KNeighborsClassifier"]=f1_score(y_test,knc.predict(X_test))"""

'grid_params = { \'n_neighbors\' : list(range(1,21,1)),\n               \'weights\' : [\'uniform\',\'distance\'],\n               \'metric\' : [\'minkowski\',\'euclidean\',\'manhattan\']}\nBest parameter value:{\'metric\': \'manhattan\', \'n_neighbors\': 18, \'weights\': \'distance\'}\nknc=KNeighborsClassifier(metric="manhattan",n_neighbors=18,weights="distance")\nknc.fit(X_train,y_train)\nModel_Scores["KNeighborsClassifier"]=f1_score(y_test,knc.predict(X_test))'

In [14]:
"""param_grid = { 'C':[0.01,0.1,0.5,1,10,100],'kernel':['rbf','poly','linear','sigmoid'],'degree':[1,2,3,4,5,6]}
Best parameter value={'C': 0.5, 'degree': 1, 'kernel': 'rbf'}
#svc=SVC(C=0.5,degree=1,kernel="rbf")
#svc.fit(X_train,y_train)
#Model_Scores["SupportVectorClassifier"]=f1_score(y_test,svc.predict(X_test))"""

'param_grid = { \'C\':[0.01,0.1,0.5,1,10,100],\'kernel\':[\'rbf\',\'poly\',\'linear\',\'sigmoid\'],\'degree\':[1,2,3,4,5,6]}\nBest parameter value={\'C\': 0.5, \'degree\': 1, \'kernel\': \'rbf\'}\n#svc=SVC(C=0.5,degree=1,kernel="rbf")\n#svc.fit(X_train,y_train)\n#Model_Scores["SupportVectorClassifier"]=f1_score(y_test,svc.predict(X_test))'

In [15]:
"""param_grid={'penalty':['l2'],'loss':['squared_hinge','hinge'],'C':[0.01,0.03,0.05,1,10]}
Best parameter value={'C': 0.03, 'loss': 'squared_hinge', 'penalty': 'l2'}
lsvc=LinearSVC(C=0.03,loss="squared_hinge",penalty="l2",max_iter=15000)
lsvc.fit(X_train,y_train)
Model_Scores["LinearSVC"]=f1_score(y_test,lsvc.predict(X_test))"""

'param_grid={\'penalty\':[\'l2\'],\'loss\':[\'squared_hinge\',\'hinge\'],\'C\':[0.01,0.03,0.05,1,10]}\nBest parameter value={\'C\': 0.03, \'loss\': \'squared_hinge\', \'penalty\': \'l2\'}\nlsvc=LinearSVC(C=0.03,loss="squared_hinge",penalty="l2",max_iter=15000)\nlsvc.fit(X_train,y_train)\nModel_Scores["LinearSVC"]=f1_score(y_test,lsvc.predict(X_test))'

In [16]:
"""param_grid={'criterion':['entropy','gini'],'splitter':['random','best'],'min_samples_leaf':list(range(10,20,2)),'max_depth':list(range(2,14,1)),'min_samples_split':list(range(1,31,1))}
Best parameter value={'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 14, 'min_samples_split': 28, 'splitter': 'random'}
dtc=DecisionTreeClassifier(criterion='gini',max_depth=7,min_samples_leaf=14,min_samples_split=28,splitter='random')
dtc.fit(X_train,y_train)
Model_Scores["DecisionTreeClassifier"]=f1_score(y_test,dtc.predict(X_test))"""

'param_grid={\'criterion\':[\'entropy\',\'gini\'],\'splitter\':[\'random\',\'best\'],\'min_samples_leaf\':list(range(10,20,2)),\'max_depth\':list(range(2,14,1)),\'min_samples_split\':list(range(1,31,1))}\nBest parameter value={\'criterion\': \'gini\', \'max_depth\': 7, \'min_samples_leaf\': 14, \'min_samples_split\': 28, \'splitter\': \'random\'}\ndtc=DecisionTreeClassifier(criterion=\'gini\',max_depth=7,min_samples_leaf=14,min_samples_split=28,splitter=\'random\')\ndtc.fit(X_train,y_train)\nModel_Scores["DecisionTreeClassifier"]=f1_score(y_test,dtc.predict(X_test))'

In [17]:
"""param_grid={'n_estimators' : [1200,1300,1400],'max_samples' : [250,275,300],'max_features' : [28,29,30]}
Best parameter value={'max_features': 29, 'max_samples': 250, 'n_estimators': 1400}
bc=BaggingClassifier(max_samples=250,n_estimators=1400)
bc.fit(X_train,y_train)
Model_Scores["BaggingClassifier"]=f1_score(y_test,bc.predict(X_test))"""

'param_grid={\'n_estimators\' : [1200,1300,1400],\'max_samples\' : [250,275,300],\'max_features\' : [28,29,30]}\nBest parameter value={\'max_features\': 29, \'max_samples\': 250, \'n_estimators\': 1400}\nbc=BaggingClassifier(max_samples=250,n_estimators=1400)\nbc.fit(X_train,y_train)\nModel_Scores["BaggingClassifier"]=f1_score(y_test,bc.predict(X_test))'

In [18]:
"""rfc=RandomForestClassifier(n_estimators=800,min_samples_split=10,min_samples_leaf=2,max_depth=50,random_state=10)
rfc.fit(X_train,y_train)
Model_Scores["RandomForestClassifier"]=f1_score(y_test,rfc.predict(X_test))"""

'rfc=RandomForestClassifier(n_estimators=800,min_samples_split=10,min_samples_leaf=2,max_depth=50,random_state=10)\nrfc.fit(X_train,y_train)\nModel_Scores["RandomForestClassifier"]=f1_score(y_test,rfc.predict(X_test))'

In [19]:
rfc2=RandomForestClassifier(n_estimators=800,min_samples_split=10,min_samples_leaf=2,max_depth=50,random_state=10)
rfc2.fit(demo,demo_y)

RandomForestClassifier(max_depth=50, min_samples_leaf=2, min_samples_split=10,
                       n_estimators=800, random_state=10)

In [20]:
adc2=AdaBoostClassifier(random_state=10,learning_rate=0.04,n_estimators=100)
adc2.fit(demo,demo_y)

AdaBoostClassifier(learning_rate=0.04, n_estimators=100, random_state=10)

In [21]:
"""gbc=GradientBoostingClassifier(random_state=10,learning_rate=0.005,n_estimators=100,subsample=0.8,max_depth=8,min_samples_split=100,min_samples_leaf=20)
gbc.fit(X_train,y_train)
Model_Scores["GradientBoostingClassifier"]=f1_score(y_test,gbc.predict(X_test))"""

'gbc=GradientBoostingClassifier(random_state=10,learning_rate=0.005,n_estimators=100,subsample=0.8,max_depth=8,min_samples_split=100,min_samples_leaf=20)\ngbc.fit(X_train,y_train)\nModel_Scores["GradientBoostingClassifier"]=f1_score(y_test,gbc.predict(X_test))'

In [22]:
"""adc=AdaBoostClassifier(random_state=10,learning_rate=0.04,n_estimators=100)
adc.fit(X_train,y_train)
Model_Scores["AdaBoostClassifier"]=f1_score(y_test,adc.predict(X_test))"""

'adc=AdaBoostClassifier(random_state=10,learning_rate=0.04,n_estimators=100)\nadc.fit(X_train,y_train)\nModel_Scores["AdaBoostClassifier"]=f1_score(y_test,adc.predict(X_test))'

In [23]:
"""mlp=MLPClassifier(random_state=10,max_iter=1000)
mlp.fit(X_train,y_train)
print(f1_score(y_test,mlp.predict(X_test)))"""

'mlp=MLPClassifier(random_state=10,max_iter=1000)\nmlp.fit(X_train,y_train)\nprint(f1_score(y_test,mlp.predict(X_test)))'

In [24]:
"""mlp1=MLPClassifier(random_state=10,max_iter=1000,activation="tanh")
param_grid = {
    'hidden_layer_sizes': [(200,150,100)],
    'solver': ['sgd'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
mlp_gs=GridSearchCV(mlp1,param_grid,cv=5,n_jobs=-1)
mlp_gs.fit(X_train,y_train)"""

'mlp1=MLPClassifier(random_state=10,max_iter=1000,activation="tanh")\nparam_grid = {\n    \'hidden_layer_sizes\': [(200,150,100)],\n    \'solver\': [\'sgd\'],\n    \'alpha\': [0.0001, 0.05],\n    \'learning_rate\': [\'constant\',\'adaptive\'],\n}\nmlp_gs=GridSearchCV(mlp1,param_grid,cv=5,n_jobs=-1)\nmlp_gs.fit(X_train,y_train)'

In [25]:
"""XGBC=xgb.XGBClassifier(random_state=10,gamma=0.3,n_estimators=100,max_depth=6,min_child_weight=2,learning_rate=0.05,colsample_bytree=0.75,subsample=0.45,scale_pos_weight=1,reg_alpha=1e-6)
XGBC.fit(X_train,y_train)
print(f1_score(y_test,XGBC.predict(X_test)))"""

'XGBC=xgb.XGBClassifier(random_state=10,gamma=0.3,n_estimators=100,max_depth=6,min_child_weight=2,learning_rate=0.05,colsample_bytree=0.75,subsample=0.45,scale_pos_weight=1,reg_alpha=1e-6)\nXGBC.fit(X_train,y_train)\nprint(f1_score(y_test,XGBC.predict(X_test)))'

In [26]:
"""XGBC1=xgb.XGBClassifier(random_state=10,gamma=0.3,n_estimators=100,max_depth=6,min_child_weight=2,learning_rate=0.05,colsample_bytree=0.75,subsample=0.45,scale_pos_weight=1,reg_alpha=1e-6)
params = {
    'reg_alpha':[1e-5,1e-4,1e-6,1e-7]
}
search = GridSearchCV(XGBC1,params, cv=5, n_jobs=-1)
search.fit(X_train,y_train)"""

"XGBC1=xgb.XGBClassifier(random_state=10,gamma=0.3,n_estimators=100,max_depth=6,min_child_weight=2,learning_rate=0.05,colsample_bytree=0.75,subsample=0.45,scale_pos_weight=1,reg_alpha=1e-6)\nparams = {\n    'reg_alpha':[1e-5,1e-4,1e-6,1e-7]\n}\nsearch = GridSearchCV(XGBC1,params, cv=5, n_jobs=-1)\nsearch.fit(X_train,y_train)"

In [27]:
test=pd.read_csv('/kaggle/input/marketing-strategy-personalised-offer/test_data.csv')

In [28]:
for i in test.columns:
    if test[i].dtypes=="object":
        test[i]=imputer.fit_transform(np.array(test[i]).reshape(-1,1))
    if test[i].dtypes=="int64":
        test[i]=imputer2.fit_transform(np.array(test[i]).reshape(-1,1))
test.pop('travelled_more_than_5mins_for_offer')

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
5300    1.0
5301    1.0
5302    1.0
5303    1.0
5304    1.0
Name: travelled_more_than_5mins_for_offer, Length: 5305, dtype: float64

In [29]:
for i in test.columns:
    if test[i].dtypes=="object":
        test[i]=ordenc.fit_transform(np.array(test[i]).reshape(-1,1))

In [30]:
test=StandardScaler().fit_transform(test)

In [31]:
prediction= rfc2.predict(test)
arr=[]
for i in prediction:
    if i==0:
        arr.append('No')
    if i==1:
        arr.append('Yes')
submission=pd.DataFrame(columns = ['id','Offer Accepted'])
submission['id']= [i for i in range(len(arr))]
submission['Offer Accepted'] = arr
submission.to_csv('submission.csv',index=False)

In [32]:
print("\n".join("{}\t\t{}".format(k, v) for k, v in Model_Scores.items()))

In [33]:
"""DummyClassifier		0.5213379469434833
LogisticRegression		0.685749844430616
RidgeClassifier		0.6824851701529816
GaussianNaiveBayes		0.6844583987441131
KNeighborsClassifier		0.6684210526315788
SupportVectorClassifier		0.7035236938031592
LinearSVC		0.6855541718555417
DecisionTreeClassifier		0.6717607973421926
BaggingClassifier		0.7112068965517242
RandomForestClassifier		0.7152276295133437
GradientBoostingClassifier		0.7247269672360683
AdaBoostClassifier		0.7229969920700028"""

'DummyClassifier\t\t0.5213379469434833\nLogisticRegression\t\t0.685749844430616\nRidgeClassifier\t\t0.6824851701529816\nGaussianNaiveBayes\t\t0.6844583987441131\nKNeighborsClassifier\t\t0.6684210526315788\nSupportVectorClassifier\t\t0.7035236938031592\nLinearSVC\t\t0.6855541718555417\nDecisionTreeClassifier\t\t0.6717607973421926\nBaggingClassifier\t\t0.7112068965517242\nRandomForestClassifier\t\t0.7152276295133437\nGradientBoostingClassifier\t\t0.7247269672360683\nAdaBoostClassifier\t\t0.7229969920700028'